In [31]:
# libraries 
import pandas as pd 
import os
import numpy as np
import time
import random
from csv import reader
import time

# plots:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


# scraping
from bs4 import BeautifulSoup
from lxml import etree #for using XPATH with beautifulsoup
import requests
import numpy as np

#JSON
import json

# GeoJSON
# import geopandas as gpd

# regular expression
import re 

# concurrent futures - boosts the process of scraping utilicing the CPU better
# two main classes- 
#       1) executor class: manages all the threads and workload
#       2) futures class: creates a little instance and manages data coming back
import concurrent.futures

# libraries: 
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import glob 

import ast

.

# ------------------------ Read in data from step 4 (merged files) called "Boliga_merged_files.csv"
path: D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_4_Merge_files

.

In [2]:
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_4_Merge_files\Boliga_merged_files.csv'

mylist = []

for chunk in  pd.read_csv(path, encoding='utf-8',low_memory=False, chunksize=20000):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis= 0)
del mylist

In [3]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031595 entries, 0 to 1031594
Data columns (total 48 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   unitId                  1031595 non-null  object 
 1   isSalesValid            1031595 non-null  bool   
 2   isActive                1031595 non-null  bool   
 3   addressString           1031595 non-null  object 
 4   gadeString              1031595 non-null  object 
 5   hus_nr                  1031595 non-null  object 
 6   etagebetegn             270092 non-null   object 
 7   side_Doernr             215293 non-null   object 
 8   zipString               1031595 non-null  object 
 9   zipCodeString           1031595 non-null  int64  
 10  imageUrl                417998 non-null   object 
 11  isApartment             1031595 non-null  bool   
 12  propertyType            1031595 non-null  int64  
 13  municipalityCode        1031595 non-null  int64  
 14  es

.

# --------> DROP useless columns which are (11 columns): 

* imageUrl
* isBlocked
* isInSalesAgent
* isInWatchlist
* watchlistNote
* canGetVR
* relatedImages
* propDataSettings
* propDataSettingTypes
* boligAfgift
* soldAgentInfo


.

In [252]:
drop_columns = [
    'imageUrl',
    'isBlocked',
    'isInSalesAgent',
    'isInWatchlist',
    'watchlistNote',
    'canGetVR',
    'relatedImages',
    'propDataSettings',
    'propDataSettingTypes',
    'boligAfgift',
    'soldAgentInfo'
]

big_data.drop(columns=drop_columns, inplace=True)
big_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031595 entries, 0 to 1031594
Data columns (total 38 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   unitId                  1031595 non-null  object 
 1   isSalesValid            1031595 non-null  bool   
 2   isActive                1031595 non-null  bool   
 3   addressString           1031595 non-null  object 
 4   gadeString              1031595 non-null  object 
 5   hus_nr                  1031595 non-null  object 
 6   etagebetegn             270092 non-null   object 
 7   side_Doernr             215293 non-null   object 
 8   zipString               1031595 non-null  object 
 9   zipCodeString           1031595 non-null  int64  
 10  isApartment             1031595 non-null  bool   
 11  propertyType            1031595 non-null  int64  
 12  municipalityCode        1031595 non-null  int64  
 13  esrCode                 1031595 non-null  int64  
 14  la

.

# ------------> drop obesrvations (properties) that belong to co-operative housing association (similar to sweden)

.

In [253]:
big_data['isCoop'].value_counts()

isCoop
False    1031507
True          88
Name: count, dtype: int64

In [257]:
big_data = big_data[big_data['isCoop']==False]
len(big_data)

1031507

In [ ]:
### then drop that column: 
big_data.drop(columns=['isCoop'], inplace=True)

In [266]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1031507 entries, 0 to 1031594
Data columns (total 36 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   unitId                  1031507 non-null  object 
 1   isSalesValid            1031507 non-null  bool   
 2   isActive                1031507 non-null  bool   
 3   addressString           1031507 non-null  object 
 4   gadeString              1031507 non-null  object 
 5   hus_nr                  1031507 non-null  object 
 6   etagebetegn             270077 non-null   object 
 7   side_Doernr             215286 non-null   object 
 8   zipString               1031507 non-null  object 
 9   zipCodeString           1031507 non-null  int64  
 10  isApartment             1031507 non-null  bool   
 11  propertyType            1031507 non-null  int64  
 12  municipalityCode        1031507 non-null  int64  
 13  esrCode                 1031507 non-null  int64  
 14  lat    

.

# -----> Rename Columns: 

* Three categories 
    * ID columns called:
        * "RowID_....
    * Address columns called:
        * Address_Info_....
    * Features called: 
        * Feature_....
    * dictionary columns called:
        * RAW_dictionary_....


.


In [279]:
big_data['addressFloor'].unique()

array([nan, 5.0, 3.0, 2.0, 1.0, 0.0, 4.0, 9.0, 8.0, 10.0, 16.0, 7.0, 12.0,
       -1.0, 6.0, 14.0, 13.0, 11.0, 27.0, 24.0, 21.0, 18.0, 22.0, 35.0,
       36.0, 33.0, 25.0, 23.0, 15.0, 31.0, 39.0, 28.0, 19.0, 17.0, 43.0,
       20.0, 40.0, 26.0, 37.0, 32.0, 30.0, 38.0, 34.0, 29.0, 41.0, 42.0,
       '0.0', '1.0', '-1.0', '2.0', '4.0', '6.0', '3.0', '23.0', '13.0',
       '12.0', '5.0', '9.0', '8.0', '17.0', '7.0', '11.0', '10.0', '14.0',
       '1', '3', '4', '0', '2', '5', '-1', '6', '7', '16', '8', '9', '11',
       '13', '10', '12', 'k2'], dtype=object)

In [280]:
new_column_names = { 
    'unitId' : 'RowID_MAIN_boliga_ROW_ID_unitID', 
    'isActive' : 'Feature_dummy_property_is_on_sale',   
    'addressString' : 'Address_info_addressString', 
    'gadeString' : 'Address_info_streetString', 
    'hus_nr' : 'Address_info_houseNumber', 
    'etagebetegn' : 'Address_info_floorString', 
    'side_Doernr' : 'Address_info_sideDoorNumber', 
    'zipString' : 'Address_info_zipString_code_and_name', 
    'zipCodeString' : 'Address_info_zipCode',  
    'isApartment' : 'Feature_dummy_isApartment',   
    'propertyType' : 'Feature_categorical_propertyTypeNumber',  
    'municipalityCode' : 'RowID_municipalityCode',  
    'esrCode' : 'RowID_propertyCode_esrCode',  
    'lat' : 'GEO_latitude',
    'lon' : 'GEO_longitude',
    'addressFloor' : 'Feature_Floor_level_of_property', 
    'ejendomsværdiskat' : 'Feature_property_value_Tax_ejendomsværdiskat',
    'grundskyld' : 'Feature_Land_Tax',
    'propertyTypeName' : 'Feature_category_propertyTypeName', 
    'bbrInfoBox' : 'RAW_dictionary_bbrInfoBox', 
    'buildingInfo' : 'RAW_dictionary_buildingInfo', 
    'unitInfo' : 'RAW_dictionary_unitInfo', 
    'ownerShipInfo' : 'RAW_dictionary_ownerShipInfo', 
    'lotInfo' : 'RAW_dictionary_lotInfo', 
    'cadastralInfos' : 'RAW_dictionary_cadastralInfos', 
    'additionalBuldingInfos' : 'RAW_dictionary_additionalBuldingInfos', 
    'floorInfos' : 'RAW_dictionary_floorInfos', 
    'additionalFloorInfos' : 'RAW_dictionary_additionalFloorInfos', 
    'salesInfos' : 'RAW_dictionary_salesInfos', 
    'evaluationInfos' : 'RAW_dictionary_evaluationInfos', 
    'esrOwnershipInfo' : 'RAW_dictionary_esrOwnershipInfo', 
    'currentEstate' : 'RAW_dictionary_currentEstate', 
    'previousEstates' : 'RAW_dictionary_previousEstates', 
    'agentInfo' : 'RAW_dictionary_agentInfo', 
    'bfenr' : 'RowID_propertyCode_bfeCode', 
    }

big_data.rename(columns=new_column_names, inplace=True)
big_data.info()

C:\Users\ABC\AppData\Local\Temp\ipykernel_8604\3069893114.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_data.rename(columns=new_column_names, inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 1031507 entries, 0 to 1031594
Data columns (total 35 columns):
 #   Column                                        Non-Null Count    Dtype  
---  ------                                        --------------    -----  
 0   RowID_MAIN_boliga_ROW_ID_unitID               1031507 non-null  object 
 1   Feature_dummy_property_is_on_sale             1031507 non-null  bool   
 2   Address_info_addressString                    1031507 non-null  object 
 3   Address_info_streetString                     1031507 non-null  object 
 4   Address_info_houseNumber                      1031507 non-null  object 
 5   Address_info_floorString                      270077 non-null   object 
 6   Address_info_sideDoorNumber                   215286 non-null   object 
 7   Address_info_zipString_code_and_name          1031507 non-null  object 
 8   Address_info_zipCode                          1031507 non-null  int64  
 9   Feature_dummy_isApartment               

.

# ----------> Re-organize the columns :

.

In [281]:
order_columns =[ 
    'RowID_MAIN_boliga_ROW_ID_unitID',               
    'RowID_propertyCode_bfeCode',                 
    'RowID_municipalityCode',                        
    'RowID_propertyCode_esrCode',      
    'Address_info_addressString',                    
    'Address_info_streetString',                     
    'Address_info_houseNumber',                      
    'Address_info_floorString',                      
    'Address_info_sideDoorNumber',                   
    'Address_info_zipString_code_and_name',          
    'Address_info_zipCode',                                              
    'GEO_latitude',                                  
    'GEO_longitude',                                 
    'Feature_dummy_isApartment',                     
    'Feature_categorical_propertyTypeNumber',     
    'Feature_Floor_level_of_property',               
    'Feature_property_value_Tax_ejendomsværdiskat',  
    'Feature_Land_Tax',                              
    'Feature_category_propertyTypeName',  
    'Feature_dummy_property_is_on_sale',             
    'RAW_dictionary_bbrInfoBox',                     
    'RAW_dictionary_buildingInfo',                   
    'RAW_dictionary_unitInfo',                       
    'RAW_dictionary_ownerShipInfo',                  
    'RAW_dictionary_lotInfo',                        
    'RAW_dictionary_cadastralInfos',                 
    'RAW_dictionary_additionalBuldingInfos',         
    'RAW_dictionary_floorInfos',                     
    'RAW_dictionary_additionalFloorInfos',           
    'RAW_dictionary_salesInfos',                     
    'RAW_dictionary_evaluationInfos',                
    'RAW_dictionary_esrOwnershipInfo',               
    'RAW_dictionary_currentEstate',                  
    'RAW_dictionary_previousEstates',                
    'RAW_dictionary_agentInfo'
    ]                      

big_data = big_data[order_columns]
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1031507 entries, 0 to 1031594
Data columns (total 35 columns):
 #   Column                                        Non-Null Count    Dtype  
---  ------                                        --------------    -----  
 0   RowID_MAIN_boliga_ROW_ID_unitID               1031507 non-null  object 
 1   RowID_propertyCode_bfeCode                    1031507 non-null  int64  
 2   RowID_municipalityCode                        1031507 non-null  int64  
 3   RowID_propertyCode_esrCode                    1031507 non-null  int64  
 4   Address_info_addressString                    1031507 non-null  object 
 5   Address_info_streetString                     1031507 non-null  object 
 6   Address_info_houseNumber                      1031507 non-null  object 
 7   Address_info_floorString                      270077 non-null   object 
 8   Address_info_sideDoorNumber                   215286 non-null   object 
 9   Address_info_zipString_code_and_name    

In [282]:
big_data['Feature_dummy_isApartment'].value_counts()

Feature_dummy_isApartment
False    853707
True     177800
Name: count, dtype: int64

In [283]:
rename_again = {
    'Feature_Land_Tax':'Feature_Land_Tax_grundskyld'              
}
big_data.rename(columns=rename_again, inplace=True)
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1031507 entries, 0 to 1031594
Data columns (total 35 columns):
 #   Column                                        Non-Null Count    Dtype  
---  ------                                        --------------    -----  
 0   RowID_MAIN_boliga_ROW_ID_unitID               1031507 non-null  object 
 1   RowID_propertyCode_bfeCode                    1031507 non-null  int64  
 2   RowID_municipalityCode                        1031507 non-null  int64  
 3   RowID_propertyCode_esrCode                    1031507 non-null  int64  
 4   Address_info_addressString                    1031507 non-null  object 
 5   Address_info_streetString                     1031507 non-null  object 
 6   Address_info_houseNumber                      1031507 non-null  object 
 7   Address_info_floorString                      270077 non-null   object 
 8   Address_info_sideDoorNumber                   215286 non-null   object 
 9   Address_info_zipString_code_and_name    

.

# ------ Create the "RAW" csv file which is the table as it is now above ! 

.

In [287]:
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\RAW_DATA_columns_renamed_and_setup\Boliga_propertySales_RAW.csv'
big_data.to_csv(save_path, encoding='utf-8' ,index=False)


.

# ---- Create "1_Base_data" csv file data 

which includes: 
* **RowID_MAIN_boliga_ROW_ID_unitID**               1031507 non-null  object 
* **RowID_propertyCode_bfeCode**                    1031507 non-null  int64  
* **RowID_municipalityCode**                        1031507 non-null  int64  
* **RowID_propertyCode_esrCode**                    1031507 non-null  int64  
* **Address_info_addressString**                    1031507 non-null  object 
* **Address_info_streetString**                     1031507 non-null  object 
* **Address_info_houseNumber**                      1031507 non-null  object 
* **Address_info_floorString**                      270077 non-null   object 
* **Address_info_sideDoorNumber**                   215286 non-null   object 
* **Address_info_zipString_code_and_name**          1031507 non-null  object 
* **Address_info_zipCode**                          1031507 non-null  int64  
* **GEO_latitude**                                  1031507 non-null  float64
* **GEO_longitude**                                 1031507 non-null  float64
* **Feature_dummy_isApartment**                     1031507 non-null  bool   
* **Feature_categorical_propertyTypeNumber**        1031507 non-null  int64  
* **Feature_Floor_level_of_property**               270077 non-null   object 
* **Feature_property_value_Tax_ejendomsværdiskat**  1031507 non-null  float64
* **Feature_Land_Tax_grundskyld**                   1031507 non-null  float64
* **Feature_category_propertyTypeName**             1031507 non-null  object 
* **Feature_dummy_property_is_on_sale**             1031507 non-null  bool   

.

In [286]:
# selected columns for the base
base_columns=[
    'RowID_MAIN_boliga_ROW_ID_unitID',
    'RowID_propertyCode_bfeCode',
    'RowID_municipalityCode',
    'RowID_propertyCode_esrCode',
    'Address_info_addressString',
    'Address_info_streetString',
    'Address_info_houseNumber',
    'Address_info_floorString',
    'Address_info_sideDoorNumber',
    'Address_info_zipString_code_and_name',
    'Address_info_zipCode',
    'GEO_latitude',
    'GEO_longitude',
    'Feature_dummy_isApartment',
    'Feature_categorical_propertyTypeNumber',
    'Feature_Floor_level_of_property',
    'Feature_property_value_Tax_ejendomsværdiskat',
    'Feature_Land_Tax_grundskyld',
    'Feature_category_propertyTypeName',
    'Feature_dummy_property_is_on_sale'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\1_Base_data\Boliga_propertySales_Base_Data.csv'
big_data[base_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 2_bbinfoBox csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* RAW_dictionary_bbrInfoBox

.

In [320]:
# selected columns for the base
bbrInfoBox_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_bbrInfoBox'
]

# Save selected columns to a CSV file
save_path_bbrinfobox = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\2_bbrInfoBox\Boliga_propertySales_bbrInfoBox.csv'
big_data[bbrInfoBox_columns].to_csv(save_path_bbrinfobox, encoding='utf-8' ,index=False)


.

# --------- Create 3_buildingInfo csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* RAW_dictionary_buildingInfo

.

In [328]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_buildingInfo'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\3_buildingInfo\Boliga_propertySales_buildingInfo.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 4_unitInfo csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_unitInfo**



.

In [329]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_unitInfo'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\4_unitInfo\Boliga_propertySales_unitInfo.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 5_ownerShipInfo csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_ownerShipInfo**

.

In [330]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_ownerShipInfo'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\5_ownerShipInfo\Boliga_propertySales_ownerShipInfo.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 6_lotInfo csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_lotInfo**

.

In [331]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_lotInfo'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\6_lotInfo\Boliga_propertySales_lotInfo.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 7_cadastralInfos csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_cadastralInfos**

.

In [332]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_cadastralInfos'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\7_cadastralInfos\Boliga_propertySales_cadastralInfos.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 8_additinalBuildingInfos csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_additionalBuldingInfos**

.

In [333]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_additionalBuldingInfos'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\8_additinalBuildingInfos\Boliga_propertySales_additinalBuildingInfos.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 9_floorInfos  csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_floorInfos**

.

In [334]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_floorInfos'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\9_floorInfos\Boliga_propertySales_floorInfos.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 10_additionalFloorInfos  csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_additionalFloorInfos**

.

In [335]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_additionalFloorInfos'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\10_additionalFloorInfos\Boliga_propertySales_additionalFloorInfos.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 11_salesInfos csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_salesInfos**

.

In [336]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_salesInfos'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\11_salesInfos\Boliga_propertySales_salesInfos.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 12_evalutationInfos csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_evaluationInfos**

.

In [337]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_evaluationInfos'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\12_evalutationInfos\Boliga_propertySales_evalutationInfos_.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 13_esrOwnershipInfo csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_esrOwnershipInfo**

.

In [338]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_esrOwnershipInfo'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\13_esrOwnershipInfo\Boliga_propertySales_esrOwnershipInfo.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 14_currentEstate csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_previousEstates**


.

In [339]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_previousEstates'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\14_currentEstate\Boliga_propertySales_currentEstate.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)


.

# --------- Create 15_agentInfo csv file that contains : 
* RowID_MAIN_boliga_ROW_ID_unitID
* RowID_propertyCode_bfeCode
* RowID_municipalityCode
* RowID_propertyCode_esrCode
* Address_info_addressString
* Address_info_streetString
* Address_info_houseNumber
* Address_info_floorString
* Address_info_sideDoorNumber
* Address_info_zipString_code_and_name
* Address_info_zipCode
* Feature_dummy_isApartment
* Feature_categorical_propertyTypeNumber
* Feature_Floor_level_of_property
* Feature_category_propertyTypeName
* Feature_dummy_property_is_on_sale
* **RAW_dictionary_agentInfo**

.

In [340]:
# selected columns for the base
sub_columns=[
'RowID_MAIN_boliga_ROW_ID_unitID',
'RowID_propertyCode_bfeCode',
'RowID_municipalityCode',
'RowID_propertyCode_esrCode',
'Address_info_addressString',
'Address_info_streetString',
'Address_info_houseNumber',
'Address_info_floorString',
'Address_info_sideDoorNumber',
'Address_info_zipString_code_and_name',
'Address_info_zipCode',
'Feature_dummy_isApartment',
'Feature_categorical_propertyTypeNumber',
'Feature_Floor_level_of_property',
'Feature_category_propertyTypeName',
'Feature_dummy_property_is_on_sale',
'RAW_dictionary_agentInfo'
]

# Save selected columns to a CSV file
save_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\15_agentInfo\Boliga_propertySales_agentInfo.csv'
big_data[sub_columns].to_csv(save_path, encoding='utf-8' ,index=False)
